# ROI ANALYSIS

Algorithm to work with this script:
- run through the first and second sections: setup the df and run modeling + plot residuals distribution
- go the R scripts for modelling (glm) and post-hoc testing (EMM) - yoou have Python implementation, but it needs to be tested
- after extracting the post-hoc data in .csv, come back to this script and run te sections for brain and box plots

Note:
- For the manuscript you only need the brain with highlighted ROIs and box plots for two task stages (collapsed over ROIs) 

In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import mne
import os
from utils import check_paths
import pandas as pd
from scipy.io import loadmat
import matplotlib.gridspec as gridspec

from mne.channels.layout import find_layout
from functools import partial
from mne.defaults import _handle_default

from mne.viz.topo import _erfimage_imshow_unified, _plot_topo

from mne.viz.utils import (
    _setup_vmin_vmax,
    add_background_image
)
from collections import namedtuple

from mne.stats import permutation_cluster_1samp_test

import scipy
from scipy.stats import zscore
pd.set_option('display.float_format', '{:.8e}'.format)

import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib qt

**MODELS LIST:**
- Model 1 - planning: _BL vs _MAIN_baseline vs _MAIN_adaptaiton (condition as a factor for each ROI)
- Model 2 - execution: _BL vs _MAIN_baseline vs _MAIN_adaptaiton (condition as a factor for each ROI)
- Model 3 - planning: _MAIN_baseline vs _MAIN_adaptaiton (condition as a factor for each ROI)
- Model 4 - execution: _MAIN_baseline vs _MAIN_adaptaiton (condition as a factor for each ROI)
- Model 5 - planning: _BL vs _MAIN_baseline vs _MAIN_adaptaiton (both condition and ROI are factors)
- Model 6 - execution: _BL vs _MAIN_baseline vs _MAIN_adaptaiton (both condition and ROI are factors)

# SET UP THE DATAFRAME WITH PAC VALUES PER CONDITION

In [2]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
src_dir = 'D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y'
src_fname = os.path.join(src_dir, 'fsaverage_bem\\bem\\fsaverage-ico4-src.fif')
src = mne.read_source_spaces(src_fname)

group = 'Y'
subs = os.listdir(os.path.join(eeg_data_dir, group))
# sub_name = 's1_pac_sub01'
# task = '_BL'
# task_stage = '_plan'
# block_name = ''


    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read


In [183]:
subs = os.listdir(os.path.join(eeg_data_dir, group))
tasks = ['_BL', '_MAIN']
task_stages = ['_plan', '_go']
blocks = ['_baseline', '_adaptation']
roi_label_names = [
    # Primary Motor Cortex
    'G_precentral-lh', 'G_precentral-rh',
    'S_central-lh', 'S_central-rh',

    # Premotor Cortex
    'S_precentral-sup-part-lh', 'S_precentral-sup-part-rh',
    'S_precentral-inf-part-lh', 'S_precentral-inf-part-rh',
    'G_front_middle-lh', 'G_front_middle-rh',

    # Supplementary Motor Area
    'G_and_S_paracentral-lh', 'G_and_S_paracentral-rh',
    'G_front_sup-lh', 'G_front_sup-rh',

    # Primary Somatosensory Cortex
    'G_postcentral-lh', 'G_postcentral-rh',
    'S_postcentral-lh', 'S_postcentral-rh',

    # Prefrontal Cortex
    'S_front_sup-lh', 'S_front_sup-rh',
    'S_front_inf-lh', 'S_front_inf-rh',
    'S_front_middle-lh', 'S_front_middle-rh'
]

rows = []

for sub, roi_label, task, task_stage in itertools.product(subs, roi_label_names, tasks, task_stages):
    if task == '_BL':
        # For task 0 → only NaN block
        rows.append([sub, roi_label, task, task_stage, np.nan, np.nan])
    else:
        # For task 1 → both block 0 and 1
        for block in blocks:
            rows.append([sub, roi_label, task, task_stage, block, np.nan])

df_pac_roi = pd.DataFrame(rows, columns=["sub", "roi", "task", "task_stage", "block", "pac_value"])
print(df_pac_roi)


               sub                roi   task task_stage        block  \
0     s1_pac_sub01    G_precentral-lh    _BL      _plan          NaN   
1     s1_pac_sub01    G_precentral-lh    _BL        _go          NaN   
2     s1_pac_sub01    G_precentral-lh  _MAIN      _plan    _baseline   
3     s1_pac_sub01    G_precentral-lh  _MAIN      _plan  _adaptation   
4     s1_pac_sub01    G_precentral-lh  _MAIN        _go    _baseline   
...            ...                ...    ...        ...          ...   
3307  s1_pac_sub77  S_front_middle-rh    _BL        _go          NaN   
3308  s1_pac_sub77  S_front_middle-rh  _MAIN      _plan    _baseline   
3309  s1_pac_sub77  S_front_middle-rh  _MAIN      _plan  _adaptation   
3310  s1_pac_sub77  S_front_middle-rh  _MAIN        _go    _baseline   
3311  s1_pac_sub77  S_front_middle-rh  _MAIN        _go  _adaptation   

      pac_value  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  
...         ...  
3307     

In [184]:
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
pac_files_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\source_pac_stats'

# read one source estimate to get the morph info
source_dir = os.path.join(eeg_data_dir, group, 's1_pac_sub01', 'preproc', 'analysis', 'source')
morphed_dir = os.path.join(source_dir, 'morphed_stcs')
id = '001'
stc = mne.read_source_estimate(os.path.join(morphed_dir, '_BL', '_plan', f's1_pac_sub01_BL_plan-stc-lcmv_epoch_{id}_morphed'))
stc_ave = stc.mean()

for i, sub_name in enumerate(subs):
    for task, task_stage in itertools.product(tasks, task_stages):
        if task == '_MAIN':
            block_names = ['_baseline', '_adaptation']
        else:
            block_names = ['']

        for block_name in block_names:
            pac_file_name = f'PAC_MI_SOURCE_{group}{task}{task_stage}{block_name}.npy'
            pac_file_path = os.path.join(pac_files_dir, pac_file_name)
            pac_all = np.load(pac_file_path)
            # pac_all.shape # (23, 5124, 20, 20)
            pac_all_ave = np.mean(pac_all, axis=(2, 3))
            # pac_all_ave.shape # (23, 5124)

            stc_ave.data = pac_all_ave[i].reshape(-1,1)
            stc_ave.data.shape # (5124, 1)

            # read roi labels
            labels = mne.read_labels_from_annot('fsaverage_bem', parc='aparc.a2009s', subjects_dir=src_dir)
            roi_labels = [lab for lab in labels if lab.name in roi_label_names]

            # extract pac values for each ROI label by averaging across vertices
            roi_tc = stc_ave.extract_label_time_course(roi_labels, src, mode='mean')
            roi_tc = roi_tc.reshape(-1)
            # roi_tc.shape # (22,)

            for roi_name, pac_value in zip(roi_label_names, roi_tc):
                cond_mask = (
                    (df_pac_roi["sub"] == sub_name) &
                    (df_pac_roi["task"] == task) &
                    (df_pac_roi["task_stage"] == task_stage) &
                    (df_pac_roi["roi"] == roi_name) 
                )
                if task == '_MAIN':
                    cond_mask &= df_pac_roi["block"].eq(block_name)
                df_pac_roi.loc[cond_mask, "pac_value"] = pac_value

df_pac_roi.to_csv(os.path.join(roi_dir, f'PAC_MI_SOURCE_{group}_ROI.csv'), index=False)

Reading labels from parcellation...
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\lh.aparc.a2009s.annot
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\rh.aparc.a2009s.annot
Extracting time courses for 24 labels (mode: mean)
Reading labels from parcellation...
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\lh.aparc.a2009s.annot
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\rh.aparc.a2009s.annot
Extracting time courses for 24 labels (mode: mean)
Reading labels from parcellation...
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\lh.aparc.a2009s.annot
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_ou

In [185]:
len(df_pac_roi["roi"].unique())

24

In [186]:
# SANITY CHECK
df_filtered = df_pac_roi[
    (df_pac_roi["sub"] == "s1_pac_sub77") &
    (df_pac_roi["task"] == "_MAIN") &
    (df_pac_roi["task_stage"] == "_go") &
    (df_pac_roi["block"] == "_adaptation")
]

print(df_filtered.to_string())

               sub                       roi   task task_stage        block      pac_value
3173  s1_pac_sub77           G_precentral-lh  _MAIN        _go  _adaptation 5.08528621e-05
3179  s1_pac_sub77           G_precentral-rh  _MAIN        _go  _adaptation 5.49439978e-05
3185  s1_pac_sub77              S_central-lh  _MAIN        _go  _adaptation 5.54547411e-05
3191  s1_pac_sub77              S_central-rh  _MAIN        _go  _adaptation 5.09607746e-05
3197  s1_pac_sub77  S_precentral-sup-part-lh  _MAIN        _go  _adaptation 5.20194824e-05
3203  s1_pac_sub77  S_precentral-sup-part-rh  _MAIN        _go  _adaptation 5.20396152e-05
3209  s1_pac_sub77  S_precentral-inf-part-lh  _MAIN        _go  _adaptation 5.75860192e-05
3215  s1_pac_sub77  S_precentral-inf-part-rh  _MAIN        _go  _adaptation 5.46427859e-05
3221  s1_pac_sub77         G_front_middle-lh  _MAIN        _go  _adaptation 5.71274182e-05
3227  s1_pac_sub77         G_front_middle-rh  _MAIN        _go  _adaptation 5.09817536e-05

___________________________________

## FITTING THE MODELS

To get the idea of the distribution (for the model):
- Very technical approach: Run random effects model > generate residuals > check distribution of the residuals > choose the model accordingly
- Empirical approach: run the model based on either gaussian distribution (identity link function) or gamma distribution (log link function) and compare which of them fits better

Steps for running stats using GLMM:
- Get the fixed effects table
- Get estimated means of conditions and difference between the means
- Run t-test based on the estmated means


What I want to compare?

1. BL_plan vs MAIN_plan_baseline vs MAIN_plan_adaptation
2. BL_go vs MAIN_go_baseline vs MAIN_go_adaptation
3. MAIN_plan_baseline vs MAIN_plan_adaptation
4. MAIN_go_baseline vs MAIN_go_adaptation


In [9]:
# Load the dataframe
group = 'Y'
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
df_pac_roi = pd.read_csv(os.path.join(roi_dir, f'PAC_MI_SOURCE_{group}_ROI.csv'))
df_pac_roi['block'] = df_pac_roi['block'].fillna('')
df_pac_roi["condition"] = df_pac_roi["task"] + df_pac_roi["block"]
df_pac_roi

,sub,roi,task,task_stage,block,pac_value,condition
0,s1_pac_sub01,G_precentral-lh,_BL,_plan,,1.42994865e-04,_BL
1,s1_pac_sub01,G_precentral-lh,_BL,_go,,1.68492354e-04,_BL
2,s1_pac_sub01,G_precentral-lh,_MAIN,_plan,_baseline,3.41858518e-04,_MAIN_baseline
3,s1_pac_sub01,G_precentral-lh,_MAIN,_plan,_adaptation,2.54340287e-04,_MAIN_adaptation
4,s1_pac_sub01,G_precentral-lh,_MAIN,_go,_baseline,1.77631731e-04,_MAIN_baseline
...,...,...,...,...,...,...,...
3307,s1_pac_sub77,S_front_middle-rh,_BL,_go,,1.26133116e-04,_BL
3308,s1_pac_sub77,S_front_middle-rh,_MAIN,_plan,_baseline,2.53442797e-04,_MAIN_baseline
3309,s1_pac_sub77,S_front_middle-rh,_MAIN,_plan,_adaptation,1.05449471e-04,_MAIN_adaptation
3310,s1_pac_sub77,S_front_middle-rh,_MAIN,_go,_baseline,1.38537406e-04,_MAIN_baseline


# CHECKING THE RESIDUALS DISTRIBUTION of null-models

In [188]:
rois = df_pac_roi['roi'].unique()
rois

array(['G_precentral-lh', 'G_precentral-rh', 'S_central-lh',
       'S_central-rh', 'S_precentral-sup-part-lh',
       'S_precentral-sup-part-rh', 'S_precentral-inf-part-lh',
       'S_precentral-inf-part-rh', 'G_front_middle-lh',
       'G_front_middle-rh', 'G_and_S_paracentral-lh',
       'G_and_S_paracentral-rh', 'G_front_sup-lh', 'G_front_sup-rh',
       'G_postcentral-lh', 'G_postcentral-rh', 'S_postcentral-lh',
       'S_postcentral-rh', 'S_front_sup-lh', 'S_front_sup-rh',
       'S_front_inf-lh', 'S_front_inf-rh', 'S_front_middle-lh',
       'S_front_middle-rh'], dtype=object)

Models 1-4

In [ ]:
import os
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.formula.api as smf

# Null / intercepts models
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
models_dir = os.path.join(roi_dir, "model_outputs")
figs_dir = os.path.join(models_dir, "figures")
check_paths(models_dir, figs_dir)

all_summaries = []

for roi in rois:
    df_roi = df_pac_roi.query("roi == @roi")
    df_plan = df_roi.query("task_stage == '_plan'")
    df_go = df_roi.query("task_stage == '_go'")
    df_plan_main = df_plan.query("task == '_MAIN'")
    df_go_main = df_go.query("task == '_MAIN'")

    print(f"\nProcessing ROI: {roi}")

    # ---------- MODEL 1 ----------
    model_1 = smf.mixedlm(
        "pac_value ~ 1", data=df_plan, groups=df_plan["sub"] # currently null model, use "pac_value ~ condition" to test condition effect
    ).fit()

    # ---------- MODEL 2 ----------
    model_2 = smf.mixedlm(
        "pac_value ~ 1", data=df_go, groups=df_go["sub"] # "pac_value ~ condition"
    ).fit()

    # ---------- MODEL 3 ----------
    model_3 = smf.mixedlm(
        "pac_value ~ 1", data=df_plan_main, groups=df_plan_main["sub"] # "pac_value ~ block"
    ).fit()

    # ---------- MODEL 4 ----------
    model_4 = smf.mixedlm(
        "pac_value ~ 1", data=df_go_main, groups=df_go_main["sub"] # "pac_value ~ block"
    ).fit()

    models = [model_1, model_2, model_3, model_4]
    model_names = [f"DF1_BL_vs_MAIN_plan_condition_{roi}", #M1_ for testing conditions
                   f"DF2_BL_vs_MAIN_go_condition_{roi}", #M2_ for testing conditions
                   f"DF3_MAIN_plan_block_{roi}", #M3_ for testing blocks
                   f"DF4_MAIN_go_block_{roi}"] #M4_ for testing blocks

    for i, (model, name) in enumerate(zip(models, model_names), start=1):
        # ---- Save model summary ----
        summary_text = model.summary().as_text()
        summary_path = os.path.join(models_dir, f"{name}_summary.txt")
        # with open(summary_path, "w") as f:
        #     f.write(summary_text)
        all_summaries.append(f"\n{'='*80}\nROI: {roi}\nModel: {name}\n{'='*80}\n")
        all_summaries.append(summary_text)

        # ---- Plot residuals ----
        residuals = model.resid
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        axes[0].hist(residuals, bins=30, edgecolor='k')
        axes[0].set_title(f"{roi}: {name[:3]} Residuals Distribution") # [:2] for testing conditions
        axes[0].set_xlabel("Residual")
        axes[0].set_ylabel("Count")

        stats.probplot(residuals, dist="norm", plot=axes[1])
        axes[1].set_title(f"{roi}: {name[:3]} Q-Q Residuals") # [:2] for testing conditions

        plot_path = os.path.join(figs_dir, f"{name}_residuals_NULL.png") # _residuals.png for testing conditions
        plt.tight_layout()
        plt.savefig(plot_path, dpi=300)
        # plt.close()

        print(f"  ✔ Saved {name} summary and residuals plot")

combined_summary_path = os.path.join(models_dir, "all_NULL_models_summary.txt") # all_models_summary.txt for testing conditions
with open(combined_summary_path, "w") as f:
    f.write("\n".join(all_summaries))

print("\n✅ All model summaries and plots saved!")


Models 5-6


In [461]:
import os
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.formula.api as smf

# Null / intercepts models
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
models_dir = os.path.join(roi_dir, "model_outputs")
figs_dir = os.path.join(models_dir, "figures")
check_paths(models_dir, figs_dir)

all_summaries = []
df_plan = df_pac_roi.query("task_stage == '_plan'")
df_go = df_pac_roi.query("task_stage == '_go'")


# ---------- MODEL 1 ----------
model_5 = smf.mixedlm(
    "pac_value ~ condition * roi", data=df_plan, groups=df_plan["sub"] # currently null model, use "pac_value ~ condition" to test condition effect
).fit()

# ---------- MODEL 2 ----------
model_6 = smf.mixedlm(
    "pac_value ~ condition * roi", data=df_go, groups=df_go["sub"] # "pac_value ~ condition"
).fit()

models = [model_5, model_6]
model_names = [f"M5_plan_condition_ROI_collapsed", #M5_ for testing plan conditions
                f"M6_go_condition_ROI_collapsed", #M6_ for testing go conditions
                ]

for i, (model, name) in enumerate(zip(models, model_names), start=1):
    # ---- Save model summary ----
    summary_text = model.summary().as_text()
    summary_path = os.path.join(models_dir, f"{name}_summary.txt")
    # with open(summary_path, "w") as f:
    #     f.write(summary_text)
    all_summaries.append(f"\n{'='*80}\nROI: {roi}\nModel: {name}\n{'='*80}\n")
    all_summaries.append(summary_text)

    # ---- Plot residuals ----
    residuals = model.resid
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].hist(residuals, bins=30, edgecolor='k')
    axes[0].set_title(f"{roi}: {name[:2]} Residuals Distribution") # [:2] for testing conditions
    axes[0].set_xlabel("Residual")
    axes[0].set_ylabel("Count")

    stats.probplot(residuals, dist="norm", plot=axes[1])
    axes[1].set_title(f"{roi}: {name[:2]} Q-Q Residuals") # [:2] for testing conditions

    plot_path = os.path.join(figs_dir, f"{name}_residuals_ROI_collapsed.png") # _residuals.png for testing conditions
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300)
    plt.close()

    print(f"  ✔ Saved {name} summary and residuals plot")

combined_summary_path = os.path.join(models_dir, "models5_6_summary.txt") # all_models_summary.txt for testing conditions
with open(combined_summary_path, "w") as f:
    f.write("\n".join(all_summaries))

print("\n✅ All model summaries and plots saved!")


d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


  ✔ Saved M5_plan_condition_ROI_collapsed summary and residuals plot
  ✔ Saved M6_go_condition_ROI_collapsed summary and residuals plot

✅ All model summaries and plots saved!


_________________________________

##  SIGNIFICANT ROI PLOTTING

In [10]:
group = 'Y'
task = '_MAIN'
task_stage = '_go'
block_name = '_adaptation'
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
pac_files_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\source_pac_stats'

# read one source estimate to get the morph info
source_dir = os.path.join(eeg_data_dir, group, 's1_pac_sub01', 'preproc', 'analysis', 'source')
morphed_dir = os.path.join(source_dir, 'morphed_stcs')
id = '001'
stc = mne.read_source_estimate(os.path.join(morphed_dir, '_BL', '_plan', f's1_pac_sub01_BL_plan-stc-lcmv_epoch_{id}_morphed'))
stc_ave = stc.mean()

pac_file_name = f'PAC_MI_SOURCE_{group}{task}{task_stage}{block_name}.npy'
pac_file_path = os.path.join(pac_files_dir, pac_file_name)
pac_all = np.load(pac_file_path)
# pac_all.shape # (23, 5124, 20, 20)
pac_all_ave = np.mean(pac_all, axis=(2, 3))
# pac_all_ave.shape # (23, 5124)

stc_ave.data = pac_all_ave[0].reshape(-1,1)
stc_ave.data.shape # (5124, 1)

# read roi labels
roi_label_names = df_pac_roi["roi"].unique()

labels = mne.read_labels_from_annot('fsaverage_bem', parc='aparc.a2009s', subjects_dir=src_dir)
roi_labels = [lab for lab in labels if lab.name in roi_label_names]

# extract pac values for each ROI label by averaging across vertices
roi_tc = stc_ave.extract_label_time_course(roi_labels, src, mode='mean')
roi_tc = roi_tc.reshape(-1)
# roi_tc.shape # (22,)


Reading labels from parcellation...
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\lh.aparc.a2009s.annot
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\rh.aparc.a2009s.annot
Extracting time courses for 24 labels (mode: mean)


In [ ]:
# roi_label_names = [
#     Primary Motor Cortex
#     'G_precentral-lh', 'G_precentral-rh',
#     'S_central-lh', 'S_central-rh',

#     # Premotor Cortex
#     'S_precentral-sup-part-lh', 'S_precentral-sup-part-rh',
#     'S_precentral-inf-part-lh', 'S_precentral-inf-part-rh',
#     'G_front_middle-lh', 'G_front_middle-rh',

#     # Supplementary Motor Area
#     'G_and_S_paracentral-lh', 'G_and_S_paracentral-rh',
#     'G_front_sup-lh', 'G_front_sup-rh',

#     # Primary Somatosensory Cortex
#     'G_postcentral-lh', 'G_postcentral-rh',
#     'S_postcentral-lh', 'S_postcentral-rh',

#     # Prefrontal Cortex
#     'S_front_sup-lh', 'S_front_sup-rh',
#     'S_front_inf-lh', 'S_front_inf-rh',
#     'S_front_middle-lh', 'S_front_middle-rh'
# ]
# # roi_label_names = df_pac_roi["roi"].unique()

# labels = mne.read_labels_from_annot('fsaverage_bem', parc='aparc.a2009s', subjects_dir=src_dir)
# roi_labels = [lab for lab in labels if lab.name in roi_label_names]

Reading labels from parcellation...
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\lh.aparc.a2009s.annot
   read 75 labels from D:\BonoKat\research project\# study 1\mri_data\fs_output\freesurfer\sub_dir\Y\fsaverage_bem\label\rh.aparc.a2009s.annot


In [36]:
## Plotting a brain with ROI labels for the manuscript
zero_data = np.zeros((stc_ave.data.shape[0], 1))
stc_ave.data = zero_data

brain = stc_ave.plot(
    subject="fsaverage_bem",
    hemi="lh",
    views=["lateral", "medial"],
    cortex="low_contrast",
    colormap="plasma",
    initial_time=0.0,
    background="white",
    colorbar=False,
)

# Add label overlay (semi-transparent color)
for roi_label in roi_labels:
    if '-lh' in roi_label.name:
        brain.add_label(roi_label, borders=True)

# Adjust brain size before saving
# brain.save_image(os.path.join(figs_dir, f'BRAIN_ROI.png'))

Using control points [0. 0. 0.]


C:\Users\a1902989\AppData\Local\Temp\ipykernel_30548\4210773505.py:5: RuntimeWarning: All data were zero
  brain = stc_ave.plot(


In [346]:
import numpy as np
import mne

# Create an empty ROI mask (same shape as stc_ave.data)
roi_mask = np.zeros_like(stc_ave.data)

n_lh = len(stc_ave.vertices[0])
n_rh = len(stc_ave.vertices[1])

for label in roi_labels:
    if label.hemi == 'lh':
        # find indices of LH vertices belonging to this ROI
        roi_vert_idx = np.where(np.isin(stc_ave.vertices[0], label.vertices))[0]
        roi_mask[roi_vert_idx, :] = 1

    elif label.hemi == 'rh':
        # find indices of RH vertices belonging to this ROI
        roi_vert_idx = np.where(np.isin(stc_ave.vertices[1], label.vertices))[0]
        roi_mask[n_lh + roi_vert_idx, :] = 1  # shift for RH


In [355]:
stc_roi_mask = mne.SourceEstimate(
    roi_mask,
    vertices=stc_ave.vertices,
    tmin=stc_ave.tmin,
    tstep=stc_ave.tstep,
    subject=stc_ave.subject,
)

brain = stc_roi_mask.plot(
    subject="fsaverage_bem",
    hemi="both",
    colormap="seismic",
    time_label="ROI mask",
    clim=dict(kind="value", lims=[0, 0, 2])
)

# Add label overlay (semi-transparent color)
for roi_label in roi_labels:
    brain.add_label(roi_label, borders=True, color='white')

In [ ]:
# Plotting ROIs with significance levels from mixed models
group = 'Y'
task = '_MAIN'
task_stage = '_go'
block_name = '_adaptation'
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
pac_files_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\source_pac_stats'

# read one source estimate to get the morph info
source_dir = os.path.join(eeg_data_dir, group, 's1_pac_sub01', 'preproc', 'analysis', 'source')
morphed_dir = os.path.join(source_dir, 'morphed_stcs')
id = '001'
stc = mne.read_source_estimate(os.path.join(morphed_dir, '_BL', '_plan', f's1_pac_sub01_BL_plan-stc-lcmv_epoch_{id}_morphed'))
stc_ave = stc.mean()

# read roi labels
roi_label_names = df_pac_roi["roi"].unique()
labels = mne.read_labels_from_annot('fsaverage_bem', parc='aparc.a2009s', subjects_dir=src_dir)
roi_labels = [lab for lab in labels if lab.name in roi_label_names]

# Path where your ROI CSVs are stored
csv_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group\\roi_source_analysis\\model_outputs'
figs_dir = os.path.join(csv_dir, "figures")

# Get hemisphere sizes
n_lh = len(stc_ave.vertices[0])
n_rh = len(stc_ave.vertices[1])

# Loop through models
for model_name in ["Model_1", "Model_2"]:
    print(f"Processing {model_name}...")

    # Initialize an empty mask for this model
    roi_mask = np.zeros((n_lh + n_rh, stc_ave.data.shape[1]))

    # Iterate through all ROI labels
    for label in roi_labels:
        roi_filename = f"ALL_MODELS_contrasts_{label.name}.csv"
        csv_path = os.path.join(csv_dir, roi_filename)

        if not os.path.exists(csv_path):
            print(f"⚠️ CSV not found for {roi_filename}, skipping.")
            continue

        # Read CSV file
        df_roi = pd.read_csv(csv_path)

        # Filter for this model
        df_model = df_roi[df_roi["model"] == model_name]
        print(df_model)

        # Count significant p-values (< 0.05)
        n_sig = np.sum(df_model["p.value"] < 0.05)

        # Assign a value depending on number of significant contrasts
        if n_sig == 3:
            value = 1.5
        elif n_sig == 2:
            value = 1.0
        elif n_sig == 1:
            value = 0.5
        else:
            value = 0.0

        # Fill in ROI vertices with that value
        if label.hemi == "lh":
            vert_idx = np.where(np.isin(stc_ave.vertices[0], label.vertices))[0]
            roi_mask[vert_idx, :] = value
        elif label.hemi == "rh":
            vert_idx = np.where(np.isin(stc_ave.vertices[1], label.vertices))[0]
            roi_mask[n_lh + vert_idx, :] = value

    # Create a new SourceEstimate for this model
    stc_roi_mask = mne.SourceEstimate(
        roi_mask,
        vertices=stc_ave.vertices,
        tmin=stc_ave.tmin,
        tstep=stc_ave.tstep,
        subject=stc_ave.subject,
    )

    if model_name == "Model_1":
        fig_title = "Planning: ROI Significance Map"
    else:
        fig_title = "Execution: ROI Significance Map"

    # Plot the result
    brain1 = stc_roi_mask.plot(
        subject="fsaverage_bem",
        hemi="lh",
        views=['lateral', 'medial'],
        colormap='plasma',
        time_label=fig_title,
        clim=dict(kind="value", lims=[0.5, 1, 1.6]),
        background="white",
        cortex= "low_contrast" # "white"
    )
    brain1.add_text(0.1, 0.9, model_name, "title", font_size=14)
    # Add label overlay (semi-transparent color)
    for roi_label in roi_labels:
        if '-lh' in roi_label.name:
            brain1.add_label(roi_label, borders=True, color='white')
    brain1.save_image(os.path.join(figs_dir, f"{fig_title.replace(':', '')}_LH.png"))

    brain2 = stc_roi_mask.plot(
        subject="fsaverage_bem",
        hemi="rh",
        views=['lateral', 'medial'],
        colormap='plasma',
        time_label=fig_title,
        clim=dict(kind="value", lims=[0.5, 1, 1.6]),
        background="white",
        cortex= "low_contrast" # "white"
    )

    brain2.add_text(0.1, 0.9, model_name, "title", font_size=14)
    # Add label overlay (semi-transparent color)
    for roi_label in roi_labels:
        if '-rh' in roi_label.name:
            brain2.add_label(roi_label, borders=True, color='white')
    brain2.save_image(os.path.join(figs_dir, f"{fig_title.replace(':', '')}_RH.png"))



Processing Model_1...
                            contrast        estimate             SE  \
0             _BL - _MAIN_adaptation  2.61483477e-05 5.86522252e-06   
1               _BL - _MAIN_baseline -1.53217481e-04 5.86522252e-06   
2  _MAIN_adaptation - _MAIN_baseline -1.79365829e-04 5.86522252e-06   

              df         t.ratio        p.value    model  
0 4.40000000e+01  4.45820216e+00 1.69065680e-04  Model_1  
1 4.40000000e+01 -2.61230466e+01 5.99433690e-28  Model_1  
2 4.40000000e+01 -3.05812487e+01 8.33817778e-31  Model_1  
                            contrast        estimate             SE  \
0             _BL - _MAIN_adaptation  3.30861180e-05 4.39851395e-06   
1               _BL - _MAIN_baseline -1.42773428e-04 4.39851395e-06   
2  _MAIN_adaptation - _MAIN_baseline -1.75859546e-04 4.39851395e-06   

              df         t.ratio        p.value    model  
0 4.40000000e+01  7.52211278e+00 5.93221454e-09  Model_1  
1 4.40000000e+01 -3.24594692e+01 6.75146073e-32  Model

d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\pyvista\core\utilities\misc.py:223: UserWarning: Encountered issue in callback (most recent call last):
  File "d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\mne\viz\_brain\_brain.py", line 1206, in _on_button_release
    self._renderer._picker.Pick(x, y, 0, self.picked_renderer)
                                         ^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Brain' object has no attribute 'picked_renderer'. Did you mean: '_picked_renderer'?
  warnings.warn(formatted_exception)


______________________________________

# BOX PLOTS

In [338]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
from statannotations.Annotator import Annotator
import matplotlib.patches as mpatches
from matplotlib import colors
import re

MODELS 1-4

In [ ]:
def sanitize_condition_filter(cond):
    """
    Extract all RHS values after '==' that are quoted (single or double quotes),
    or fallback to unquoted tokens, then join them with underscores and
    replace any non-alphanumeric characters with underscores.
    """
    # find quoted RHS values: captures the quote char and the inner text
    quoted = re.findall(r"==\s*(['\"])(.*?)\1", cond)
    if quoted:
        vals = [v for _, v in quoted]
    else:
        # fallback: capture token after == up to space or logical operator
        vals = re.findall(r"==\s*([^\s&|]+)", cond)

    # Join parts (e.g. "_MAIN" and "_plan" -> "_MAIN__plan") then clean
    joined = "_".join(v.strip().strip("'\"") for v in vals)

    # Replace any non-alphanumeric characters with underscores, collapse multiples
    s = re.sub(r'[^0-9A-Za-z]+', '_', joined)
    s = re.sub(r'_+', '_', s).strip('_')
    return s

def plot_roi_stats(roi_dir, df_pac_roi, model,
                   column_of_interest, condition_filter, 
                   hemi, figure_name,
                   palette):
    # Filter to planning stage and left-hemisphere ROIs
    df_plot = df_pac_roi.query(f"{condition_filter} and roi.str.contains({hemi})").copy()
    # print(df_plot)

    # Initialize figure
    fig, ax = plt.subplots(figsize=(len(df_plot['roi'].unique()) * 3, 5))

    # Plot boxplots
    sns.boxplot(
        data=df_plot,
        x="roi",
        y="pac_value",
        hue=column_of_interest,
        palette=palette,
        ax=ax
    )

    # ---------------------------------------------------------------------
    # ADD SIGNIFICANCE BARS PER ROI
    # ---------------------------------------------------------------------
    pairs_all = []
    pvalues_all = []

    for roi_name in df_plot["roi"].unique():
        emmeans_file = os.path.join(roi_dir, "model_outputs", f"ALL_MODELS_contrasts_{roi_name}.csv")
        df_emm = pd.read_csv(emmeans_file)
        df_emm = df_emm.query(f"model == '{model}'")
        # print(model)
        # print(df_emm)

        for _, row in df_emm.iterrows():
            conds = [c.strip() for c in row["contrast"].split("-")]
            if len(conds) == 2 and row["p.value"] <= 0.05:
                pair = [(roi_name, conds[0]), (roi_name, conds[1])]
                pairs_all.append(pair)
                pvalues_all.append(row["p.value"])
        # print(pairs_all)

    annotator = Annotator(
        ax,
        pairs=pairs_all,
        data=df_plot,
        x="roi",
        y="pac_value",
        hue=column_of_interest
    )
    annotator.set_pvalues_and_annotate(pvalues=pvalues_all)

    # ---------------------------------------------------------------------
    # Formatting
    # ---------------------------------------------------------------------
    # Set y-axis limits
    # ax.set_ylim(0, 0.0007)  # adjust to your desired range
    ax.margins(x=0.02)  # smaller = tighter, try 0 or 0.02
    ax.set_title(figure_name[:-4])
    ax.set_xlabel("ROI")
    ax.set_ylabel("PAC value")

    # Rotate ROI labels
    clean_labels = [label.get_text()[:-3] for label in ax.get_xticklabels()] # or label.get_text()
    ax.set_xticklabels(clean_labels, rotation=45, ha='right')

    # Get unique condition names from the dataframe
    conditions = df_plot[column_of_interest].unique()  # ensures consistent order
    palette_dict = dict(zip(conditions, palette))
    patches = [mpatches.Patch(color=palette_dict[cond], label=label) 
            for cond, label in zip(conditions, conditions)]
    ax.legend(
        handles=patches,
        title="Condition",
        bbox_to_anchor=(0.99, 1),
        loc='upper left',
        frameon=False
    )

    sns.despine(ax=ax)
    plt.tight_layout()
    plt.show()

    # Save the figure
    fig_path = os.path.join(roi_dir, "model_outputs", "figures", figure_name)
    fig.savefig(fig_path, dpi=300)


In [ ]:
group = 'Y'
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
df_pac_roi = pd.read_csv(os.path.join(roi_dir, f'PAC_MI_SOURCE_{group}_ROI.csv'))
df_pac_roi['block'] = df_pac_roi['block'].fillna('')
df_pac_roi["condition"] = df_pac_roi["task"] + df_pac_roi["block"]

model_column_dict = {
    'Model_1': ['condition', "task_stage == '_plan'"],
    'Model_2': ['condition', "task_stage == '_go'"],
    'Model_3': ['block', "task == '_MAIN' and task_stage == '_plan'"],
    'Model_4': ['block', "task == '_MAIN' and task_stage == '_go'"]
}

hemis = ["'-lh'", "'-rh'"]

for model, condition in model_column_dict.items():
    column_of_interest = condition[0]
    condition_filter = condition[1]
    if column_of_interest == 'block':
        palette_rgb = sns.color_palette("hls")[:2] # MAIN_baseline, MAIN_adaptation
    else:
        palette_rgb = sns.color_palette("hls")[-3:]  # BL, MAIN_baseline, MAIN_adaptation
    palette_hex = [colors.to_hex(c) for c in palette_rgb]

    for hemi in hemis:
        hemi_label = hemi.strip("'\"").replace('-', '').upper()
        token = sanitize_condition_filter(condition_filter)
        figure_name = f"PAC_{token}_ROIs_{hemi_label}_hemisphere.png"
        plot_roi_stats(roi_dir=roi_dir, df_pac_roi=df_pac_roi, model=model,
                        column_of_interest=column_of_interest, condition_filter=condition_filter,
                        hemi=hemi, palette=palette_hex,
                        figure_name=figure_name)


__________________________

MODELS 5-6

For planning:
- both condition and roi factors are significant, but no significant interaction bewteen them

For execution:
- only condition factor is significant, but not roi or interaction

Summary:
We can collapse the plot over roi and focus only on the difference between conditions (to point out the role of PAC during ADAPTATION)

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from statannotations.Annotator import Annotator

# ---------------------------------------------------------------------
# Load data
# ---------------------------------------------------------------------
group = 'Y'
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
df_pac_roi = pd.read_csv(os.path.join(roi_dir, f'PAC_MI_SOURCE_{group}_ROI.csv'))
df_pac_roi['block'] = df_pac_roi['block'].fillna('')
df_pac_roi["condition"] = df_pac_roi["task"] + df_pac_roi["block"]

df_emm_collapsed = pd.read_csv(
    os.path.join(roi_dir, 'model_outputs', f'ALL_ROI_MODELS_5_6_contrasts_collapsed.csv')
)

# ---------------------------------------------------------------------
# Model definitions
# ---------------------------------------------------------------------
model_column_dict = {
    'Model_5_condition': ['condition', "task_stage == '_plan'"],
    'Model_6_condition': ['condition', "task_stage == '_go'"],
}

# ---------------------------------------------------------------------
# Colors
# ---------------------------------------------------------------------
palette_rgb = sns.color_palette("hls", 8)[-3:]  # BL, MAIN_baseline, MAIN_adaptation
palette_hex = [colors.to_hex(c) for c in palette_rgb]

# ---------------------------------------------------------------------
# Merge both task stages into one dataframe for plotting
# ---------------------------------------------------------------------
plot_data = []
for model, condition in model_column_dict.items():
    col = condition[0]
    filter_expr = condition[1]
    df_tmp = df_pac_roi.query(filter_expr).copy()
    df_tmp["model"] = model
    plot_data.append(df_tmp)

df_plot = pd.concat(plot_data, ignore_index=True)

df_plot["mode"] = df_plot["task_stage"].map({
    "_plan": "planning",
    "_go": "execution"
})

df_plot["cond_name"] = df_plot["condition"].map({
    "_BL": "FTT",
    "_MAIN_baseline": "De-CRAT: baseline",
    "_MAIN_adaptation": "De-CRAT: adaptation"
})

# ---------------------------------------------------------------------
# Create a single boxplot: x = task_stage, hue = condition
# ---------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 6))

sns.boxplot(
    data=df_plot,
    x="mode",
    y="pac_value",
    hue="condition",
    palette=palette_hex,
    ax=ax,
    showfliers=False,
)

# ---------------------------------------------------------------------
# Build significance pairs and p-values
# ---------------------------------------------------------------------
pairs_all = []
pvalues_all = []

for model in model_column_dict.keys():
    df_emm = df_emm_collapsed.query(f"model == '{model}'")

    # Determine which stage (_plan or _go)
    stage = '_plan' if '5' in model else '_go'

    for _, row in df_emm.iterrows():
        conds = [c.strip() for c in row["contrast"].split("-")]
        if len(conds) == 2 and row["p.value"] <= 0.05:
            # Pair format for Annotator: ((x_val, hue1), (x_val, hue2))
            pair = [(stage, conds[0]), (stage, conds[1])]
            pairs_all.append(pair)
            pvalues_all.append(row["p.value"])

# ---------------------------------------------------------------------
# Annotate significance
# ---------------------------------------------------------------------
annotator = Annotator(
    ax,
    pairs=pairs_all,
    data=df_plot,
    x="task_stage",
    y="pac_value",
    hue="condition"
)
annotator.set_pvalues_and_annotate(pvalues=pvalues_all)

# ---------------------------------------------------------------------
# Beautify
# ---------------------------------------------------------------------
ax.set_title(f"PAC Comparison (collapsed over ROI) — {group} group", fontsize=14, weight='bold')
ax.set_xlabel("Task Stage")
ax.set_ylabel("PAC value")
ax.legend(title="Condition", bbox_to_anchor=(1.02, 1), loc='upper left')

conditions = df_plot['cond_name'].unique()  # ensures consistent order
palette_dict = dict(zip(conditions, palette_hex))
patches = [mpatches.Patch(color=palette_dict[cond], label=label) 
        for cond, label in zip(conditions, conditions)]
ax.legend(
    handles=patches,
    title="Condition",
    bbox_to_anchor=(0.99, 1),
    loc='upper left',
    frameon=False
)

plt.tight_layout()
plt.show()

fig_path = os.path.join(roi_dir, "model_outputs", "figures", f"PAC_{group}_group_collapsed_ROIs.png")
fig.savefig(fig_path, dpi=300)

p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

_go__BL vs. _go__MAIN_baseline: Custom statistical test, P_val:1.299e-13
_go__MAIN_baseline vs. _go__MAIN_adaptation: Custom statistical test, P_val:0.000e+00
_plan__BL vs. _plan__MAIN_baseline: Custom statistical test, P_val:0.000e+00
_plan__MAIN_baseline vs. _plan__MAIN_adaptation: Custom statistical test, P_val:0.000e+00
_go__BL vs. _go__MAIN_adaptation: Custom statistical test, P_val:0.000e+00
_plan__BL vs. _plan__MAIN_adaptation: Custom statistical test, P_val:7.441e-85


d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
C:\Users\a1902989\AppData\Local\Temp\ipykernel_2644\245235318.py:113: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument

___________________________________________

Dirty field

In [205]:
# BAR PLOT DRAFT WITH CUSTOM P-VALUES AND REMOVING NON-SIGNIFICANT COMPARISONS for future use
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
import pandas as pd

# ---------------------------------------------------------------------
# Example data
# ---------------------------------------------------------------------
df = pd.DataFrame({
    "condition": ["A", "A", "A", "B", "B", "B", "C", "C", "C"],
    "value": [3.2, 3.8, 3.5, 4.1, 4.3, 4.2, 5.1, 5.3, 5.2]
})

# Your precomputed p-values (e.g., from GLMM or emmeans)
custom_pvalues = {
    ("A", "B"): 0.012,
    ("B", "C"): 0.078,   # non-significant
    ("A", "C"): 0.001
}

# ---------------------------------------------------------------------
# Filter out non-significant pairs (p > 0.05)
# ---------------------------------------------------------------------
filtered = {pair: p for pair, p in custom_pvalues.items() if p <= 0.05}

pairs = list(filtered.keys())
pvalues = list(filtered.values())

# ---------------------------------------------------------------------
# Plot boxplots with custom color palette
# ---------------------------------------------------------------------
palette = {"A": "#1f77b4", "B": "#ff7f0e", "C": "#2ca02c"}  # distinct colors

fig, ax = plt.subplots(figsize=(6, 5))
sns.boxplot(x="condition", y="value", data=df, palette=palette, ax=ax)
# sns.stripplot(x="condition", y="value", data=df, color="black", size=5, jitter=True, ax=ax)

# ---------------------------------------------------------------------
# Annotate with numeric p-values
# ---------------------------------------------------------------------
annotator = Annotator(ax, pairs, data=df, x="condition", y="value")
annotator.set_pvalues_and_annotate(
    pvalues=pvalues
)

# ---------------------------------------------------------------------
# Styling
# ---------------------------------------------------------------------
ax.set_xlabel("Condition", fontsize=12)
ax.set_ylabel("Value", fontsize=12)
sns.despine()
plt.tight_layout()
plt.show()


p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

A vs. B: Custom statistical test, P_val:1.200e-02
A vs. C: Custom statistical test, P_val:1.000e-03


d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)


In [ ]:
# Load the PAC dataframe
group = 'Y'
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'
df_pac_roi = pd.read_csv(os.path.join(roi_dir, f'PAC_MI_SOURCE_{group}_ROI.csv'))
df_pac_roi['block'] = df_pac_roi['block'].fillna('')
df_pac_roi["condition"] = df_pac_roi["task"] + df_pac_roi["block"]
df_pac_roi

roi = 'G_precentral-lh'  # Example ROI
df_roi = df_pac_roi.query("roi == @roi")
df_plan = df_roi.query("task_stage == '_plan'")
df_go = df_roi.query("task_stage == '_go'")
df_plan_main = df_plan.query("task == '_MAIN'")
df_go_main = df_go.query("task == '_MAIN'")

# Load the emmeans dataframe by ROI
df_emmeans = pd.read_csv(os.path.join(roi_dir, 'model_outputs', f'ALL_MODELS_contrasts_{roi}.csv'))
df_emmeans

,contrast,estimate,SE,df,t.ratio,p.value,model
0,_BL - _MAIN_adaptation,2.02199443e-05,6.22157968e-06,4.40000000e+01,3.24996952e+00,6.64690446e-03,Model_1
1,_BL - _MAIN_baseline,-1.49461963e-04,6.22157968e-06,4.40000000e+01,-2.40231533e+01,1.88941676e-26,Model_1
2,_MAIN_adaptation - _MAIN_baseline,-1.69681907e-04,6.22157968e-06,4.40000000e+01,-2.72731229e+01,1.00303483e-28,Model_1
3,_BL - _MAIN_adaptation,9.20296617e-05,5.39272036e-06,4.40000000e+01,1.70655357e+01,1.52177141e-20,Model_2
4,_BL - _MAIN_baseline,-1.55481286e-05,5.39272036e-06,4.40000000e+01,-2.88316982e+00,1.82103146e-02,Model_2
5,_MAIN_adaptation - _MAIN_baseline,-1.07577790e-04,5.39272036e-06,4.40000000e+01,-1.99487055e+01,3.42242020e-23,Model_2
6,_adaptation - _baseline,-1.69681907e-04,6.08662395e-06,2.20000000e+01,-2.78778364e+01,1.17203884e-18,Model_3
7,_adaptation - _baseline,-1.07577790e-04,4.17379347e-06,2.20000000e+01,-2.57745840e+01,6.26707410e-18,Model_4


In [ ]:
model_name = 'Model_1'
df_emmeans_model = df_emmeans.query("model == @model_name")
df_emmeans_model

,contrast,estimate,SE,df,t.ratio,p.value,model
0,_BL - _MAIN_adaptation,2.02199443e-05,6.22157968e-06,4.40000000e+01,3.24996952e+00,6.64690446e-03,Model_1
1,_BL - _MAIN_baseline,-1.49461963e-04,6.22157968e-06,4.40000000e+01,-2.40231533e+01,1.88941676e-26,Model_1
2,_MAIN_adaptation - _MAIN_baseline,-1.69681907e-04,6.22157968e-06,4.40000000e+01,-2.72731229e+01,1.00303483e-28,Model_1


In [ ]:
df_plan

,sub,roi,task,task_stage,block,pac_value,condition
0,s1_pac_sub01,G_precentral-lh,_BL,_plan,,1.42994865e-04,_BL
2,s1_pac_sub01,G_precentral-lh,_MAIN,_plan,_baseline,3.41858518e-04,_MAIN_baseline
3,s1_pac_sub01,G_precentral-lh,_MAIN,_plan,_adaptation,2.54340287e-04,_MAIN_adaptation
144,s1_pac_sub07,G_precentral-lh,_BL,_plan,,1.40082769e-04,_BL
146,s1_pac_sub07,G_precentral-lh,_MAIN,_plan,_baseline,2.74261878e-04,_MAIN_baseline
...,...,...,...,...,...,...,...
3026,s1_pac_sub76,G_precentral-lh,_MAIN,_plan,_baseline,2.90965062e-04,_MAIN_baseline
3027,s1_pac_sub76,G_precentral-lh,_MAIN,_plan,_adaptation,1.12358824e-04,_MAIN_adaptation
3168,s1_pac_sub77,G_precentral-lh,_BL,_plan,,1.38532878e-04,_BL
3170,s1_pac_sub77,G_precentral-lh,_MAIN,_plan,_baseline,2.71398948e-04,_MAIN_baseline


In [ ]:
df_pac_condition = df_plan[['condition', 'pac_value']]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
palette = ["#1f77b4", "#ff7f0e", "#2ca02c"]  # distinct colors
sns.boxplot(x="condition", y="pac_value", data=df_pac_condition, palette=palette, ax=ax)

d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)


<Axes: xlabel='condition', ylabel='pac_value'>

In [ ]:
len(df_pac_roi.roi.unique())

24

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator

# -------------------------------------------------------------
# STEP 1 — Parse contrast column into tuples and extract p-values
# -------------------------------------------------------------
pval_dict = {}
for _, row in df_emmeans_model.iterrows():
    # Split by " - " and strip spaces
    cond1, cond2 = [c.strip() for c in row["contrast"].split("-")]
    pval_dict[(cond1, cond2)] = row["p.value"]

print(pval_dict)
# {'_BL', '_MAIN_adaptation'): 0.0066, ('_BL', '_MAIN_baseline'): 1.889e-26, ('_MAIN_adaptation', '_MAIN_baseline'): 1.003e-28}

# -------------------------------------------------------------
# STEP 2 — Filter significant pairs (p <= 0.05)
# -------------------------------------------------------------
filtered = {pair: p for pair, p in pval_dict.items() if p <= 0.05}

pairs = list(filtered.keys())
pvalues = list(filtered.values())

# -------------------------------------------------------------
# STEP 3 — Create boxplot and annotate
# -------------------------------------------------------------
df_pac_condition = df_plan[["condition", "pac_value"]]
fig, ax = plt.subplots(figsize=(6, 5))
palette = ["#1f77b4", "#ff7f0e", "#2ca02c"]
sns.boxplot(x="condition", y="pac_value", data=df_pac_condition, palette=palette, ax=ax)
ax.set_title(roi[:-3])
# sns.stripplot(x="condition", y="pac_value", data=df_pac_condition, color="black", size=4, jitter=True, ax=ax)

annotator = Annotator(ax, pairs, data=df_pac_condition, x="condition", y="pac_value")
annotator.set_pvalues_and_annotate(pvalues=pvalues)

sns.despine()
plt.tight_layout()
plt.show()


{('_BL', '_MAIN_adaptation'): 0.017127082542224, ('_BL', '_MAIN_baseline'): 7.97484627320158e-26, ('_MAIN_adaptation', '_MAIN_baseline'): 6.245233381385089e-28}
p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

_BL vs. _MAIN_baseline: Custom statistical test, P_val:7.975e-26
_MAIN_baseline vs. _MAIN_adaptation: Custom statistical test, P_val:6.245e-28
_BL vs. _MAIN_adaptation: Custom statistical test, P_val:1.713e-02


d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
from statannotations.Annotator import Annotator
import matplotlib.patches as mpatches
from matplotlib import colors

group = 'Y'
roi_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\roi_source_analysis'

# Filter to planning stage and left-hemisphere ROIs
df_plot = df_pac_roi.query("task == '_MAIN' and task_stage == '_plan' and roi.str.contains('-lh')").copy()

palette = ["#1f77b4", "#ff7f0e", "#2ca02c"]

# Initialize figure
fig, ax = plt.subplots(figsize=(len(df_plot['roi'].unique()) * 3, 5))

# Plot boxplots
sns.boxplot(
    data=df_plot,
    x="roi",
    y="pac_value",
    hue="block",
    palette=palette,
    ax=ax
)

# ---------------------------------------------------------------------
# ADD SIGNIFICANCE BARS PER ROI
# ---------------------------------------------------------------------
pairs_all = []
pvalues_all = []

for roi_name in df_plot["roi"].unique():
    emmeans_file = os.path.join(roi_dir, "model_outputs", f"ALL_MODELS_contrasts_{roi_name}.csv")
    df_emm = pd.read_csv(emmeans_file)
    df_emm = df_emm.query("model == 'Model_3'")

    for _, row in df_emm.iterrows():
        conds = [c.strip() for c in row["contrast"].split("-")]
        if len(conds) == 2 and row["p.value"] <= 0.05:
            pair = [(roi_name, conds[0]), (roi_name, conds[1])]
            pairs_all.append(pair)
            pvalues_all.append(row["p.value"])

annotator = Annotator(
    ax,
    pairs=pairs_all,
    data=df_plot,
    x="roi",
    y="pac_value",
    hue="block"
)
annotator.set_pvalues_and_annotate(pvalues=pvalues_all)

# ---------------------------------------------------------------------
# Formatting
# ---------------------------------------------------------------------
ax.margins(x=0.02)  # smaller = tighter, try 0 or 0.02
ax.set_title("PAC across conditions and ROIs")
ax.set_xlabel("ROI")
ax.set_ylabel("PAC value")

# Rotate ROI labels
clean_labels = [label.get_text()[:-3] for label in ax.get_xticklabels()]
ax.set_xticklabels(clean_labels, rotation=45, ha='right')

# Get unique condition names from the dataframe
conditions = df_plot["block"].unique()  # ensures consistent order
palette_dict = dict(zip(conditions, palette))
patches = [mpatches.Patch(color=palette_dict[cond], label=label) 
        for cond, label in zip(conditions, conditions)]
ax.legend(
    handles=patches,
    title="Condition",
    bbox_to_anchor=(0.99, 1),
    loc='upper left',
    frameon=False
)


sns.despine(ax=ax)
plt.tight_layout()
plt.show()

# Save the figure
# fig_path = os.path.join(roi_dir, "model_outputs", "figures", "PAC_plan_ROIs_LEFT_hemisphere.png")
# fig.savefig(fig_path, dpi=300)


C:\Users\a1902989\AppData\Local\Temp\ipykernel_2644\588996506.py:20: UserWarning: The palette list has more values (10) than needed (2), which may not be intended.
  sns.boxplot(
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
d:\BonoKat\research project\motor_pac\venv\Lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)


p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

S_front_sup-lh__baseline vs. S_front_sup-lh__adaptation: Custom statistical test, P_val:1.191e-18
S_front_inf-lh__baseline vs. S_front_inf-lh__adaptation: Custom statistical test, P_val:5.334e-17
S_front_middle-lh__baseline vs. S_front_middle-lh__adaptation: Custom statistical test, P_val:9.876e-17
S_postcentral-lh__baseline vs. S_postcentral-lh__adaptation: Custom statistical test, P_val:1.752e-16
S_central-lh__baseline vs. S_central-lh__adaptation: Custom statistical test, P_val:1.528e-15
G_precentral-lh__baseline vs. G_precentral-lh__adaptation: Custom statistical test, P_val:1.172e-18
S_precentral-sup-part-lh__baseline vs. S_precentral-sup-part-lh__adaptation: Custom statistical test, P_val:2.251e-17
S_precentral-inf-part-lh__baseline vs. S_precentral-inf-part-lh__adaptation: Custom statistica

C:\Users\a1902989\AppData\Local\Temp\ipykernel_2644\588996506.py:67: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(clean_labels, rotation=45, ha='right')
